In [1]:
import langchain

In [2]:
from getpass import getpass 

In [24]:
HUGGINGFACEHUB_API_TOKEN = getpass()

 ········


In [26]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

# Huggingface API test

In [27]:
# from https://huggingface.co/docs/api-inference/quicktour

import json
import requests
API_URL = "https://api-inference.huggingface.co/models/gpt2"
headers = {"Authorization": f"Bearer {HUGGINGFACEHUB_API_TOKEN}"}
def query(payload):
    data = json.dumps(payload)
    response = requests.request("POST", API_URL, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))
data = query("Can you please let us know more details about your ")

In [28]:
data

[{'generated_text': 'Can you please let us know more details about your iphone?\n\nWe are currently working'}]

# Langchain example using API key

In [39]:
#repo_id = "google/flan-t5-xl"
repo_id = "google/flan-t5-base"


In [40]:
llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":0, "max_length":64})

In [38]:
print(llm)

HuggingFaceHub
Params: {'repo_id': 'google/flan-t5-xl', 'task': None, 'model_kwargs': {'temperature': 0, 'max_length': 64}}


In [32]:
from langchain import PromptTemplate, LLMChain, HuggingFaceHub

In [41]:
template = """Question: {question}

Answer; Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain= LLMChain(prompt=prompt, llm=llm)

In [34]:
question = "Who won the FIFA World Cup in the year 1994?"

In [37]:
print(prompt)

input_variables=['question'] output_parser=None partial_variables={} template="Question: {question}\n\nAnswer; Let's think step by step." template_format='f-string' validate_template=True


In [42]:
answer = llm_chain.run(question)

In [43]:
print(answer)

FIFA World Cup was a football tournament played in the United States. FIFA World Cup was held in 1994. The answer: finland.


# Example using downloaded models

Note this does not yet work on a MacMini b/c HuggingFacePipeline requires a CUDA enabled version of pytorch, which is not default for Mac.  

In [44]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM


In [45]:
model_id = 'google/flan-t5-small'

In [46]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [47]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /Users/asif/.pyenv/versions/analysis/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary /Users/asif/.pyenv/versions/analysis/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...
dlopen(/Users/asif/.pyenv/versions/analysis/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so, 0x0006): tried: '/Users/asif/.pyenv/versions/analysis/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/asif/.pyenv/versions/analysis/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (no such file), '/Users/asif/.pyenv/version

/Users/asif/.pyenv/versions/analysis/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [48]:
pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=128
)

In [49]:
local_llm = HuggingFacePipeline(pipeline=pipeline)

In [51]:
# teet w/o langchain
#print(local_llm("What is the capital of California?"))